# Apend New IDs

I just ran a database update, I now have a set of SRX/SRR that need to be added to the datastore. I am going to figure out the best way to do that here.

In [1]:
import os
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../lib')
from ncbi_remap.config import DATA_STORE
from ncbi_remap.io import HDFStore
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

# Setup notebook
nbconfig = Nb.setup_notebook()

# Connect to data store
store = HDFStore(DATA_STORE, mode='a')

Please check output/fbgn2chrom.tsv. If it does not exist, run bin/fbgn2chrom.py
last updated: 2018-02-16 
Git hash: 52de887b1e498cf97f88ca6e19510826d15109e4


In [2]:
store.root

/ (RootGroup) ''
  children := ['aln' (Group), 'ids' (Group), 'layout' (Group), 'prealn' (Group), 'strand' (Group), 'test' (Group)]

In [3]:
def get_update_db_ids():
    from pymongo import MongoClient
    try:
        with open('../output/.mongodb_host', 'r') as fh:
            host = fh.read().strip()
    except FileNotFoundError:
        host = 'localhost'

    mongoClient = MongoClient(host=host, port=27022)
    db = mongoClient['sra']
    ncbi = db['ncbi']

    # Dump all ids out of database
    df = pd.DataFrame(list(ncbi.aggregate([
        {
            '$unwind': '$sra.run'
        },
        {
            '$match': {
                'sra.run.run_id': {'$exists': 1}
            }
        },
        {
            '$project': {
                '_id': 0,
                'srx': '$_id',
                'srr': '$sra.run.run_id'
            }
        },
    ])))

    return df.set_index(['srx', 'srr'])

In [4]:
curr_sra_ids = get_update_db_ids()

In [16]:
store.update_ids_from_db(curr_sra_ids)

../lib/ncbi_remap/io.py:121: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['flag_abi_solid', 'flag_alignment_bad', 'flag_complete', 'flag_download_bad', 'flag_quality_scores_bad', 'flag_qc_passed', 'flag_merge']]

  self._append_new_ids(key, new_ids)
../lib/ncbi_remap/io.py:122: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['flag_abi_solid', 'flag_alignment_bad', 'flag_complete', 'flag_download_bad', 'flag_quality_scores_bad', 'flag_qc_passed', 'flag_merge']]

  self._drop_removed_ids(key, removed_ids)
../lib/ncbi_remap/io.py:147: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->axis1] [items->None]

  self._store[key] = 

KeyboardInterrupt: 

In [45]:
store.root.prealn.workflow

/prealn/workflow (Group) ''
  children := ['bamtools_stats' (Group), 'collectrnaseqmetrics' (Group), 'fastq' (Group), 'fastq_screen' (Group), 'feature_counts' (Group), 'hisat2' (Group), 'markduplicates' (Group), 'merge' (Group), 'samtools_idxstats' (Group), 'samtools_stats' (Group)]

In [35]:
store['prealn/complete'] = store['prealn/complete'].set_index(['srx', 'srr'])

In [49]:
store['prealn/workflow/bamtools_stats'].replace('NA', np.nan)

,,'Proper-pairs',Both pairs mapped,Duplicates,Failed QC,Forward strand,Mapped reads,Paired-end reads,Percent Duplicates,Percent Failed QC,Percent Forward,Percent Mapped,Percent Paired-end,Percent Reverse,Read 1,Read 2,Reverse strand,Singletons,Total reads
srx,srr,,,,,,,,,,,,,,,,,,
SRX2581987,SRR5278103,4077654.0,4176200.0,0.0,0.0,33130867.0,4882229.0,35575122.0,0.0,0.0,93.129314,13.723717,100.0,6.870686,17819863.0,17755259.0,2444255.0,706029.0,35575122.0
SRX2556775,SRR5251041,NaN,NaN,0.0,0.0,1816742.0,836972.0,0.0,0.0,0.0,80.562662,37.115172,0.0,19.437338,NaN,NaN,438325.0,NaN,2255067.0
SRX2556774,SRR5251040,NaN,NaN,0.0,0.0,703013.0,199959.0,0.0,0.0,0.0,87.390950,24.856734,0.0,12.609050,NaN,NaN,101433.0,NaN,804446.0
SRX2556773,SRR5251039,NaN,NaN,0.0,0.0,2478819.0,936806.0,0.0,0.0,0.0,83.757835,31.654123,0.0,16.242165,NaN,NaN,480688.0,NaN,2959507.0
SRX2556772,SRR5251038,NaN,NaN,0.0,0.0,2279118.0,706474.0,0.0,0.0,0.0,86.593287,26.841921,0.0,13.406713,NaN,NaN,352862.0,NaN,2631980.0
SRX2556771,SRR5251037,NaN,NaN,0.0,0.0,1946169.0,765281.0,0.0,0.0,0.0,83.471969,32.823209,0.0,16.528031,NaN,NaN,385355.0,NaN,2331524.0
SRX2556770,SRR5251036,NaN,NaN,0.0,0.0,2098848.0,599597.0,0.0,0.0,0.0,87.290295,24.937013,0.0,12.709705,NaN,NaN,305598.0,NaN,2404446.0
SRX2541782,SRR5234256,NaN,NaN,0.0,0.0,13651088.0,22578624.0,0.0,0.0,0.0,57.105017,94.450546,0.0,42.894983,NaN,NaN,10254146.0,NaN,23905234.0
SRX2541781,SRR5234255,NaN,NaN,0.0,0.0,23466076.0,34956840.0,0.0,0.0,0.0,61.119196,91.047773,0.0,38.880804,NaN,NaN,14927878.0,NaN,38393954.0
